This will compute the 0 - 29 hop embeddings for the in sample data frames

This creates 30 (~9k,10k) .csv files in `MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/more_buckets_data/{index}_hop_larger_embedding.csv` where index refers to the hop number.

I only expect to use the first few ~10ish dataframes as features since the rest get a lot of random noise. I am including up to 30 since I want to only have to do it once.

I have also included the time costs of each of the steps.

In [7]:
!git clone https://github.com/parkerburchett/pysmiles # pysmiles just with te
!pip install pyTDC
!git clone https://github.com/parkerburchett/TDC-DeepLearning

Cloning into 'pysmiles'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 420 (delta 12), reused 18 (delta 4), pack-reused 390
Receiving objects: 100% (420/420), 134.29 KiB | 22.38 MiB/s, done.
Resolving deltas: 100% (251/251), done.
Cloning into 'TDC-DeepLearning'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 245 (delta 83), reused 208 (delta 47), pack-reused 0
Receiving objects: 100% (245/245), 48.62 MiB | 22.28 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Checking out files: 100% (97/97), done.


In [8]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
from pysmiles.pysmiles import read_smiles
from tdc.single_pred import ADME
import os
os.chdir('/content/TDC-DeepLearning/')
from utils import ColorRefinement as cr # this is the graph embedding Algo I wrote
from sklearn.decomposition import PCA
import lightgbm as lgb


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from tdc import utils
from tdc.benchmark_group import admet_group

group = admet_group(path = 'data/')
benchmark = group.get('cyp2c9_veith')
training_smiles = benchmark['train_val']['Drug']

Found local copy...


In [11]:
%%time 
graphs = [read_smiles(s) for s in training_smiles]

CPU times: user 14 s, sys: 199 ms, total: 14.2 s
Wall time: 14.1 s


In [ ]:
%%time
hop_feature_dfs = cr.create_hop_feature_dfs(graphs,
                                        num_hops=30, # means 30 dataframes
                                        num_colors=10_000) # means 10k columns.

In [ ]:
%%time
def add_labels_to_df(df):
  df.columns = [f'color_{i}' for i in range(len(df.columns))]
  df.index.rename('index', inplace=True)
  return df

for index, df in enumerate(hop_feature_dfs):
  save_location =f'/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/more_buckets_data/{index}_hop_larger_embedding.csv'
  df = add_labels_to_df(df)
  df.to_csv(save_location)
  print(f'saved df {index}')
